# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [1]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys
from collections import deque  

## 2. INICIALIZAR MEDIAPIPE

In [2]:
## 2. INICIALIZAR MEDIAPIPE HOLISTIC (SIN FACE)
mp_holistic = mp.solutions.holistic
mp_draw = mp.solutions.drawing_utils

dataset_dir = "dataset_12"
model_path = "gesture_model_me_12.h5"
sequence_length = 30
total_landmarks = 225
gestures = []
X_mean = None
X_std = None

num_camara = 0

## 3. FUNCIONES PRINCIPALES

In [3]:
# Funciones principales
def init_system():
    global gestures
    os.makedirs(dataset_dir, exist_ok=True)
    gestures = get_existing_gestures()
    
def get_existing_gestures():
    return [d for d in os.listdir(dataset_dir) 
           if os.path.isdir(os.path.join(dataset_dir, d))]

## 4. DETECCION DE MANO

In [4]:
def detect_pose_and_hands():
    print("\nIniciando detección de pose y manos. Presiona 'ESC' para salir.")
    cap = cv2.VideoCapture(0)

    # Inicializamos Holistic
    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=0,  # Menor complejidad para velocidad
        min_detection_confidence=0.45,
        min_tracking_confidence=0.45,
        refine_face_landmarks=False  # No refinamos la cara
    ) as holistic:

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb_frame)

            # Dibujar manos y pose (ignorar cara)
            if results.left_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.pose_landmarks:
                mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            cv2.imshow("Detección de Manos", frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

## 5. RECOLLECION DE DATOS 

In [5]:
def collect_data():
    global gestures
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))
    
    save_dir = os.path.join(dataset_dir, gesture)
    os.makedirs(save_dir, exist_ok=True)

    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")
    print("Mantenga la seña frente a la cámara...")
    
    cap = cv2.VideoCapture(0)
    sequence = []
    counter = 0

    with mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=0,
        min_detection_confidence=0.45,
        min_tracking_confidence=0.45,
        refine_face_landmarks=False
    ) as holistic:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(rgb_frame)

            all_landmarks = []
            # Mano izquierda: 21 puntos
            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])
            else:
                all_landmarks.extend([0.0] * 63)
            # Mano derecha: 21 puntos
            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])
            else:
                all_landmarks.extend([0.0] * 63)
            # Pose: 33 puntos
            if results.pose_landmarks:
                for lm in results.pose_landmarks.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])
            else:
                all_landmarks.extend([0.0] * 99)

            # Solo se guarda si se obtuvo la dimensión correcta
            if len(all_landmarks) == total_landmarks:
                sequence.append(all_landmarks)

            # Dibujar los landmarks (manos y pose)
            if results.left_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.pose_landmarks:
                mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            if len(sequence) == sequence_length:
                np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)
                counter += 1
                sequence = []
                print(f"Secuencias capturadas: {counter}/{num_sequences}")

            cv2.imshow("Recolección de Datos", frame)
            if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:
                break

    cap.release()   
    cv2.destroyAllWindows()
    gestures = get_existing_gestures()
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")

## 6. CARGA DE DATOS

In [6]:
def load_data():
    X = []
    y = []
    
    for label_idx, gesture in enumerate(gestures):
        gesture_dir = os.path.join(dataset_dir, gesture)
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            sequence = np.load(seq_path)
            
            if sequence.shape == (sequence_length, total_landmarks):
                X.append(sequence)
                y.append(label_idx)
    
    return np.array(X), np.array(y), gestures


## 7. ENTRENAMIENTO DEL MODELO

In [7]:
def train_model():
    global X_mean, X_std, gestures
    
    # 1. Verificar datos de entrenamiento
    gestures = get_existing_gestures()
    if not gestures:
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return

    # 2. Cargar y preparar datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data()
    y = tf.keras.utils.to_categorical(y)

    # 3. Calcular parámetros de normalización
    X_mean = np.mean(X, axis=(0, 1))
    X_std = np.std(X, axis=(0, 1))
    X = (X - X_mean) / X_std  # Normalización

    # 4. Guardar parámetros de normalización
    np.savez('normalization_params.npz', mean=X_mean, std=X_std)
    
    # 5. Arquitectura optimizada del modelo
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # 6. Compilación y entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # 7. Guardar modelo y resultados
    model.save(model_path)
    print(f"\nModelo guardado en {model_path}")
    
    # 8. Conversión a TFLite con configuraciones especiales
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False
    
    try:
        tflite_model = converter.convert()
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")
    
    # Mostrar métricas finales
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [8]:
def convert_to_tflite():
    try:
        # Cargar el modelo entrenado
        model = tf.keras.models.load_model(model_path)
        
        # Configurar el conversor con parámetros especiales
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Añadir estas 3 líneas clave para compatibilidad con LSTM
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter._experimental_lower_tensor_list_ops = False
        converter.allow_custom_ops = True  # Permitir operaciones personalizadas
        
        # Realizar la conversión
        tflite_model = converter.convert()
        
        # Guardar el modelo cuantizado
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
            
        print("\n✅ Conversión a TFLite exitosa!")
        
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")
        print("Posibles soluciones:")
        print("1. Verifique que el modelo .h5 existe")
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")
        print("3. Reinicie el runtime/kernel")

def representative_dataset_gen():
    # Generador de datos de ejemplo para calibración
    for _ in range(100):
        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]

## 8. EVALUACION DEL MODELO

In [9]:
from threading import Thread
from collections import deque

def evaluate():
    if not os.path.exists("model_quantized.tflite"):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    
    # 1. Cargar parámetros y modelo
    try:
        with np.load('normalization_params.npz') as data:
            X_mean = data['mean']
            X_std = data['std']
            
        interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]  # Mantener [0]
        output_details = interpreter.get_output_details()[0]
    except Exception as e:
        print(f"\nError crítico: {str(e)}")
        return

    # 2. Configuración de cámara
    cap = cv2.VideoCapture(0)  # Forzar backend DirectShow
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    
    if not cap.isOpened():
        print("\n¡No se puede acceder a la cámara!")
        return

    # 3. Variables de estado
    current_gesture = None  # Variable para almacenar la última seña detectada
    current_confidence = 0.0
    sequence = deque(maxlen=sequence_length)  # Usar deque
    frame_counter = 0


        # Inicializamos Holistic para evaluación
    holistic = mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=0,
        min_detection_confidence=0.45,
        min_tracking_confidence=0.45,
        refine_face_landmarks=False
    )
    
    # 4. Bucle principal de evaluación
    while True:
        ret, frame = cap.read()
        if not ret:
            print("\nError en captura de frame")
            break
            
        frame_counter += 1
        
        # Procesar cada 2 frames para optimizar
        if frame_counter % 2 != 0:
            continue
            
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb_frame)
        # Extraer landmarks de manos y pose
        if results:
            landmarks = []
            # Mano izquierda
            if results.left_hand_landmarks:
                for lm in results.left_hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            else:
                landmarks.extend([0.0] * 63)
            # Mano derecha
            if results.right_hand_landmarks:
                for lm in results.right_hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            else:
                landmarks.extend([0.0] * 63)
            # Pose
            if results.pose_landmarks:
                for lm in results.pose_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            else:
                landmarks.extend([0.0] * 99)
            
            # Asegurarse de tener la dimensión correcta
            if len(landmarks) == total_landmarks:
                sequence.append(landmarks)
            
            # Dibujar landmarks en pantalla
            if results.left_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_draw.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.pose_landmarks:
                mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            
            # Realizar predicción cada 30 frames (aproximadamente 1 seg)
            if len(sequence) == sequence_length and frame_counter % 30 == 0:
                try:
                    seq_array = np.array(sequence)
                    seq_array = (seq_array - X_mean) / (X_std + 1e-7)
                    input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                    
                    print("Forma de entrada:", input_data.shape)
                    
                    interpreter.set_tensor(input_details['index'], input_data)
                    interpreter.invoke()
                    prediction = interpreter.get_tensor(output_details['index'])[0]
                    
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[predicted_idx]
                    
                    if confidence > 0.75:
                        current_gesture = gestures[predicted_idx]
                        current_confidence = confidence
                        print("Nueva seña detectada:", current_gesture)
                    
                    cv2.putText(frame, f"{current_gesture} ({current_confidence:.2%})", 
                                (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
                    
                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None
                            
                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None  # Resetear si hay error
                    
        if current_gesture:
            cv2.putText(frame, 
              f"{current_gesture} ({current_confidence:.2%})", 
              (10, 50), 
              cv2.FONT_HERSHEY_SIMPLEX, 
              1, 
              (0, 255, 0), 
              2)

        # Mostrar frame reducido
        cv2.imshow("Predicciones", cv2.resize(frame, (640, 480)))
        
        if cv2.waitKey(1) & 0xFF == 27: break
        
        # 4.8 Salir con ESC
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 5. Limpieza
    cap.release()
    cv2.destroyAllWindows()
    print("\nSistema de evaluación detenido")

## 9. REENTRENAR GESTO

In [10]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## 10. MENU

In [11]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Evaluar")
        print("5. Reentrenar Gesto")
        print("6. Convertir a TFLite")  # Nueva opción
        print("7. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_pose_and_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':
            retrain_gesture()
        elif choice == '6':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '7':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [12]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. Convertir a TFLite
7. Salir
Forma de entrada: (1, 30, 225)
Nueva seña detectada: HOLA}
Forma de entrada: (1, 30, 225)
Nueva seña detectada: HOLA}
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Nueva seña detectada: CONSTRUIR
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Nueva seña detectada: CONSTRUIR
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Nueva seña detectada: COMO ESTAS
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: (1, 30, 225)
Forma de entrada: